In [35]:
# librería Natural Language Toolkit, usada para trabajar con textos
import nltk

# librería para manejar las flexiones gramaticales en el idioma español.
import spacy

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import numpy as np
import sys
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer ###
from unidecode import unidecode



# Punkt permite separar un texto en frases.
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')



from statistics import mode
from scipy import stats as st


import ydata_profiling as dp
from num2words import num2words
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt


import json

import joblib



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orteg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\orteg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\orteg\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\orteg\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [36]:
# Uso de la libreria pandas para la lectura de archivos
data = pd.read_csv('tipo1_entrenamiento_estudiantes.csv', sep=',', encoding = 'utf-8')
textos = data.copy()
datos_limpios = data.copy()

In [37]:
def eliminar_duplicados(Pdataframe, Psubset):
    Pdataframe = Pdataframe.drop_duplicates(subset=Psubset)
    Pdataframe.reset_index(drop=True, inplace=True)
    return Pdataframe

def convertir_minusculas(texto):
    """Convierte un string a minúsculas"""
    return texto.lower()

def convertir_enteros_a_texto(texto):
    """Convierte los números enteros en su versión textual en español en un texto dado."""
    def reemplazar(match):
        numero = int(match.group(0))
        return num2words(numero, lang='es')

    texto_convertido = re.sub(r'\b\d+\b', reemplazar, texto)
    return texto_convertido

def remove_non_ascii(text):
    """Remove non-ASCII characters while preserving the structure of the text"""
    cleaned_text = ""
    for char in text:
        if ord(char) < 128:  # Si el carácter es ASCII
            cleaned_text += char  # Conserva el carácter
        else:
            cleaned_text += unicodedata.normalize('NFKD', char).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            # Reemplaza el carácter no ASCII con su equivalente ASCII ignorando los no convertibles
    return cleaned_text

def eliminar_caracteres_especiales(texto):
    """Elimina los caracteres especiales como \r, \n, etc., de un texto."""
    texto_limpio = re.sub(r'[\r\n\t]', ' ', texto)
    # La expresión regular r'[\r\n\t]' coincide con \r, \n y \t
    return texto_limpio



datos_limpios.reset_index(drop=True, inplace=True)

# Load Spanish language model
nlp = spacy.load("es_core_news_lg")

def lemmatize_spanish_text(text):
    # Process the text
    doc = nlp(text)
    # Lemmatize each token and return the lemmatized text
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text

# Batch processing with DataFrame.apply
batch_size = 500
num_batches = (len(datos_limpios) - 1) // batch_size + 1  # Adjusted to ensure all rows are processed

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(datos_limpios))
    datos_limpios.loc[start_idx:end_idx, "Review"] = datos_limpios.loc[start_idx:end_idx, "Review"].apply(lemmatize_spanish_text)
    print(f"Processed batch {i+1}/{num_batches}")

# Process the remaining rows if any
start_idx = num_batches * batch_size
if start_idx < len(datos_limpios):
    datos_limpios.loc[start_idx:, "Review"] = datos_limpios.loc[start_idx:, "Review"].apply(lemmatize_spanish_text)


def eliminar_tildes_y_puntuacion(texto):
    # Eliminar tildes y acentos
    texto_sin_tildes = unidecode(texto)
    
    # Eliminar signos de puntuación
    texto_sin_puntuacion = texto_sin_tildes.translate(str.maketrans('', '', string.punctuation))
    
    return texto_sin_puntuacion

stop_words = set(stopwords.words('spanish'))
more_stopwords = ['ser','estar', 'tener', 'haber']
stop_words = stop_words.union(more_stopwords)
def remove_stopwords(text):
    tokens = nltk.word_tokenize(text)  # Tokenización de la reseña
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]  # Filtrado de stopwords
    return ' '.join(filtered_tokens)  # Reconstruir la reseña sin stopwords

Processed batch 1/16
Processed batch 2/16
Processed batch 3/16
Processed batch 4/16
Processed batch 5/16
Processed batch 6/16
Processed batch 7/16
Processed batch 8/16
Processed batch 9/16
Processed batch 10/16
Processed batch 11/16
Processed batch 12/16
Processed batch 13/16
Processed batch 14/16
Processed batch 15/16
Processed batch 16/16


In [38]:
datos_limpios.to_csv("Datos_limpios.csv", index=False)


In [39]:
datos_limpios.to_csv("Datos_limpios.csv", index=False)

data_2 = pd.read_csv("particion_prueba_estudiantes.csv", sep=",", encoding="utf-8")
pipeline = Pipeline([
    ("duplicates", eliminar_duplicados),
    ("minusculas", convertir_minusculas),
    ("numeros enteros", convertir_enteros_a_texto),
    ("ascii", remove_non_ascii),
    ("caracteres_especiales", eliminar_caracteres_especiales),
    ("puntuacion", eliminar_tildes_y_puntuacion),
    ("stopwords", remove_stopwords)
])
datos_limpios_2 = data_2.copy()
datos_limpios_2 = eliminar_duplicados(datos_limpios_2, ["Review"])
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(convertir_minusculas)
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(convertir_enteros_a_texto)
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(remove_non_ascii)
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(eliminar_caracteres_especiales)
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(eliminar_tildes_y_puntuacion)
datos_limpios_2['Review'] = datos_limpios_2['Review'].apply(remove_stopwords)

In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Supongamos que datos_limpios es un DataFrame con las columnas Review y Class
data_set = datos_limpios.copy()
data_test = datos_limpios_2.copy()

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
x = int(len(data_set) * 0.8)
train, test = data_set.iloc[:x, :], data_set.iloc[x:, :]

# Separar las características (Review) y las etiquetas (Class)
X_train, y_train = train['Review'], train['Class']
X_test, y_test = test['Review'], test['Class']

# Convertir texto a características numéricas utilizando TF-IDF
vectorizer = TfidfVectorizer(max_features=2500)  # Utilizamos solo las 1000 características más importantes
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_data_test = vectorizer.transform(data_test["Review"])

# Entrenar un modelo de regresión logística
model = LogisticRegression()
model.fit(X_train_vec, y_train)

# Predecir las etiquetas para el conjunto de prueba
y_pred = model.predict(X_test_vec)

class_prediction = model.predict(X_data_test)
data_test["Class"] = class_prediction

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", round(accuracy, 5)*100, "%")

print("Classification Report:\n", classification_report(y_test, y_pred))

data_test.to_csv("particion_prueba_estudiantes_predicted.csv", index=False)

Precisión del modelo: 50.54 %
Classification Report:
               precision    recall  f1-score   support

           1       0.53      0.37      0.43       160
           2       0.47      0.38      0.42       245
           3       0.39      0.42      0.41       288
           4       0.45      0.44      0.45       409
           5       0.61      0.72      0.66       473

    accuracy                           0.51      1575
   macro avg       0.49      0.47      0.47      1575
weighted avg       0.50      0.51      0.50      1575

